In [1]:
#Necessary imports for plotting, pandas, math operations, and multiple print statements.
import pandas as pd
import numpy as np

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
# read cleaned dataset
df = pd.read_csv("datasets/movies.csv")

# get only necessary columns 
main_df = df[["originalTitle", "Scripts", "genres"]]
main_df = main_df[main_df["Scripts"].str.contains("<NA>") == False].reset_index().drop("index", axis=1)
main_df

,originalTitle,Scripts,genres
0,Kate & Leopold,time time proposed fourth dimension yet mortal...,"Comedy,Fantasy,Romance"
1,Barton Fink,kissing goodbye four walls six flights six fli...,"Comedy,Drama,Thriller"
2,Bill & Ted's Bogus Journey,time reached second crucial turning point dest...,"Adventure,Comedy,Fantasy"
3,Billy Bathgate,think makes cheap dago move man took coll man ...,"Crime,Drama,Thriller"
4,The Butcher's Wife,water across sea little island lived young gir...,"Comedy,Fantasy,Romance"
...,...,...,...
2010,Hickok,keep goddamn head right hear take gunner stay ...,Western
2011,Lost in London,venom 5 million poker tourney download go thin...,"Comedy,Drama"
2012,A Bad Moms Christmas,ill blue without ill blue alright white ill bl...,Comedy
2013,SPF-18,real life movies little mixed ask penny cooper...,"Drama,Romance,Sport"


In [3]:
# tf idf 
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy.sparse

tfidf = TfidfVectorizer()
response = tfidf.fit_transform(main_df["Scripts"])
feature_names = tfidf.get_feature_names()
vectorized_df = pd.DataFrame(response.toarray()).set_axis(feature_names, axis=1)
vectorized_df

,00,000,0000,00000,0002,000500010000,001,0010,0012,0013,...,zoology,zoom,zoomed,zooming,zooms,zoos,zounds,zucchini,zwieback,zygote
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2011,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2012,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2013,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
# samp = vectorized_df.copy()
# samp.insert(0, "movie_titles", main_df["originalTitle"])
# samp = samp.set_index("movie_titles")
# samp.sample(n=5, axis=1).sample(n=5, axis=0)

In [5]:
# edit vectorized dataframe
vectorized_df.insert(0, "movie_genres", main_df["genres"].copy())
vectorized_df.insert(0, "movie_titles", main_df["originalTitle"].copy())

# divide into train and test
vectorized_train = vectorized_df.loc[:1800].copy()
vectorized_test = vectorized_df.loc[1801:].copy()

In [6]:
# splitting genres only for train
vectorized_train["movie_genres"] = vectorized_train["movie_genres"].squeeze().str.split(",")
vectorized_train = vectorized_train.explode("movie_genres")
vectorized_train

,movie_titles,movie_genres,00,000,0000,00000,0002,000500010000,001,0010,...,zoology,zoom,zoomed,zooming,zooms,zoos,zounds,zucchini,zwieback,zygote
0,Kate & Leopold,Comedy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0
0,Kate & Leopold,Fantasy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0
0,Kate & Leopold,Romance,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0
1,Barton Fink,Comedy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0
1,Barton Fink,Drama,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1799,The Huntsman: Winter's War,Action,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0
1799,The Huntsman: Winter's War,Adventure,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0
1799,The Huntsman: Winter's War,Drama,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0
1800,Joe,Crime,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.02024,0.0,0.0


In [7]:
# get 250 movies for each genre (Drama, Comedy, Action, Crime, Romance, Adventure, Thriller)
vectorized_train["movie_genres"].value_counts()
sorted_df = vectorized_train.sort_values(by=['movie_genres'], ascending=True)

not_remove = {'Action', 'Adventure', 'Comedy', 'Crime', 'Drama',  'Romance', 'Thriller'}
remove = {'Mystery', 'Fantasy', 'Sci-Fi', 'Biography', 'Animation', 'Family', 'Horror',  'Sport', 'Music', 'War', 'History', 'Musical', 'Western', 'Documentary'}

sorted_df = sorted_df[~sorted_df["movie_genres"].isin(remove)]

vectorized_train = vectorized_train.iloc[0:0]
for x in not_remove:
    temp = sorted_df.loc[sorted_df["movie_genres"] == x]
    vectorized_train = vectorized_train.append(temp.iloc[:300])

# get list of movies used for training (so we dont use them in testing)
train_movies = list(set(vectorized_train["movie_titles"].tolist()))

vectorized_train
vectorized_train["movie_genres"].value_counts()

Drama        1023
Comedy        741
Action        512
Crime         430
Romance       384
Adventure     360
Thriller      325
Mystery       181
Fantasy       144
Sci-Fi        131
Biography      95
Animation      94
Family         83
Horror         65
Sport          57
Music          44
History        14
War            14
Musical        12
Western         9
Name: movie_genres, dtype: int64

,movie_titles,movie_genres,00,000,0000,00000,0002,000500010000,001,0010,...,zoology,zoom,zoomed,zooming,zooms,zoos,zounds,zucchini,zwieback,zygote
1553,Exodus: Gods and Kings,Adventure,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
530,Charlie's Angels,Adventure,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1710,Noah,Adventure,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1219,Speed Racer,Adventure,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
933,Pirates of the Caribbean: Dead Man's Chest,Adventure,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1415,Limitless,Thriller,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1410,Escape Plan,Thriller,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.018827,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1406,Jack Ryan: Shadow Recruit,Thriller,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
460,Very Bad Things,Thriller,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Adventure    300
Romance      300
Crime        300
Comedy       300
Drama        300
Action       300
Thriller     300
Name: movie_genres, dtype: int64

In [8]:
# convert "genres" from categorical to quantitative
d = {'Comedy': 0, 'Fantasy': 1, 'Romance': 2, 'Drama': 3, 'Thriller': 4, 'Action': 5, 'Crime': 6, 'Adventure': 7, 
     'Family': 8, 'Mystery': 9, 'Biography': 10, 'Music': 11, 'Horror': 12, 'Sci-Fi': 13, 'Animation': 14,  
     'Sport': 15, 'History': 16,  'Western': 17, 'War': 18,  'Musical': 19, 'Documentary': 20
}

vectorized_train["movie_genres"] = vectorized_train["movie_genres"].map(d)
vectorized_train

,movie_titles,movie_genres,00,000,0000,00000,0002,000500010000,001,0010,...,zoology,zoom,zoomed,zooming,zooms,zoos,zounds,zucchini,zwieback,zygote
1553,Exodus: Gods and Kings,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
530,Charlie's Angels,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1710,Noah,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1219,Speed Racer,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
933,Pirates of the Caribbean: Dead Man's Chest,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1415,Limitless,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1410,Escape Plan,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.018827,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1406,Jack Ryan: Shadow Recruit,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
460,Very Bad Things,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
# remove movies that are in train set from test set
test_movies = list(set(vectorized_test["movie_titles"].tolist()))
for x in list(set(test_movies).intersection(train_movies)):
    vectorized_test = vectorized_test[vectorized_test["movie_titles"] != x]

vectorized_test

,movie_titles,movie_genres,00,000,0000,00000,0002,000500010000,001,0010,...,zoology,zoom,zoomed,zooming,zooms,zoos,zounds,zucchini,zwieback,zygote
1801,Mothers and Daughters,Drama,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1802,Avengers: Age of Ultron,"Action,Adventure,Sci-Fi",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.017077,0.0,0.0
1803,The Homesman,"Drama,Western",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1804,Smurfs: The Lost Village,"Adventure,Animation,Comedy",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1805,They Came Together,"Comedy,Romance",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2010,Hickok,Western,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
2011,Lost in London,"Comedy,Drama",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
2012,A Bad Moms Christmas,Comedy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
2013,SPF-18,"Drama,Romance,Sport",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0


In [10]:
# convert movie_genres in test data to a list of numerized genres
vectorized_test_temp = vectorized_test.copy()
vectorized_test_temp["movie_genres"] = vectorized_test_temp["movie_genres"].squeeze().str.split(",")
vectorized_test_temp = vectorized_test_temp.explode("movie_genres")
vectorized_test_temp["movie_genres"] = vectorized_test_temp["movie_genres"].map(d)
vectorized_test_temp = vectorized_test_temp.groupby("movie_titles")["movie_genres"].apply(list).reset_index()
vectorized_test_temp
vectorized_test["movie_genres"] = vectorized_test_temp["movie_genres"].values
vectorized_test

,movie_titles,movie_genres
0,5 Flights Up,"[0, 3]"
1,A Bad Moms Christmas,[0]
2,A Fighting Man,"[3, 15]"
3,A Little Chaos,"[3, 2]"
4,A Million Ways to Die in the West,"[0, 17]"
...,...,...
207,Wonder Wheel,"[3, 2]"
208,Wonderstruck,"[7, 3, 9]"
209,X-Men: Apocalypse,"[5, 7, 13]"
210,Yoga Hosers,"[5, 0, 1]"


,movie_titles,movie_genres,00,000,0000,00000,0002,000500010000,001,0010,...,zoology,zoom,zoomed,zooming,zooms,zoos,zounds,zucchini,zwieback,zygote
1801,Mothers and Daughters,"[0, 3]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1802,Avengers: Age of Ultron,[0],0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.017077,0.0,0.0
1803,The Homesman,"[3, 15]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1804,Smurfs: The Lost Village,"[3, 2]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1805,They Came Together,"[0, 17]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2010,Hickok,"[3, 2]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
2011,Lost in London,"[7, 3, 9]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
2012,A Bad Moms Christmas,"[5, 7, 13]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
2013,SPF-18,"[5, 0, 1]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0


In [11]:
# knn 

from sklearn.neighbors import KNeighborsClassifier 
from numpy import dot
from numpy.linalg import norm
import math

X_train = vectorized_train.copy().drop(["movie_titles", "movie_genres"], axis=1)
y_train = vectorized_train["movie_genres"]
X_test = vectorized_test.copy().drop(["movie_titles", "movie_genres"], axis=1)
y_test = vectorized_test["movie_genres"]

# X_train
# X_test
# y_train
# y_test

def cos_sim(x, y):
    return dot(x, y) / (norm(x) * norm(y))

k = int(math.sqrt(X_train.shape[0]))

# knn_model = KNeighborsClassifier(n_neighbors=10, weights='distance', metric=cos_sim)
knn_model = KNeighborsClassifier(n_neighbors=15)
knn_model.fit(X=X_train, y=y_train)

KNeighborsClassifier(n_neighbors=15)

In [12]:
test_preds = knn_model.predict(X_test)
test_preds

# printing out neighbors and their distances
# distances, indices = knn_model.kneighbors(X_test)
# distances

array([2, 5, 6, 0, 2, 6, 0, 2, 6, 4, 4, 5, 6, 2, 0, 0, 0, 2, 0, 6, 5, 7,
       4, 0, 2, 2, 0, 7, 3, 0, 2, 0, 6, 0, 2, 6, 0, 6, 0, 0, 5, 2, 2, 2,
       0, 0, 3, 0, 0, 7, 0, 6, 0, 0, 7, 3, 2, 0, 0, 5, 0, 0, 3, 0, 0, 6,
       6, 3, 3, 2, 3, 0, 0, 0, 6, 0, 6, 0, 0, 6, 0, 0, 2, 3, 2, 4, 0, 3,
       7, 7, 0, 4, 2, 2, 0, 5, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 4, 0, 3, 6,
       2, 0, 7, 6, 0, 3, 6, 6, 0, 2, 7, 0, 4, 0, 2, 0, 2, 0, 0, 4, 0, 0,
       3, 2, 0, 0, 0, 0, 6, 6, 6, 0, 0, 0, 2, 6, 4, 3, 2, 0, 2, 3, 0, 6,
       2, 6, 5, 0, 0, 0, 2, 4, 4, 4, 0, 3, 6, 6, 0, 3, 0, 2, 0, 0, 3, 0,
       3, 4, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 3, 0, 0, 0, 6, 2, 0,
       2, 2, 0, 0, 0, 6, 0, 0, 0, 3, 2, 0, 0, 2], dtype=int64)

In [13]:
pd.set_option('display.max_rows', None)
# evaluate the performance of our model

# compute the error percentage of two arrays
# actual is an array of arrays of numbers
# predicted is an array of numbers
# return the number of correct predictions/total
def error_perc(actual, predicted):
    correct = 0;
    for x, y in zip(actual, predicted):
        for i in x:
            if y == i:
                correct = correct + 1
                # print(x, y)
                # print(str(correct) + ", " + str(len(actual)))
    return correct / len(actual)

ep = error_perc(y_test, test_preds)
ep

compare = vectorized_test[["movie_titles","movie_genres"]]
compare = compare.assign(predicted = pd.Series(test_preds).values)
compare


0.2688679245283019

,movie_titles,movie_genres,predicted
1801,Mothers and Daughters,"[0, 3]",2
1802,Avengers: Age of Ultron,[0],5
1803,The Homesman,"[3, 15]",6
1804,Smurfs: The Lost Village,"[3, 2]",0
1805,They Came Together,"[0, 17]",2
1806,Tokarev,"[7, 3, 8]",6
1807,Dark Places,"[5, 6, 3]",0
1808,Carol,"[3, 4]",2
1809,The Family,"[7, 8, 1]",6
1810,The Magnificent Seven,"[10, 6, 3]",4
